In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import tqdm

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import exp
from scipy.optimize import curve_fit

import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
import hdf5storage
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.cm as cm

from utility_simulated_drift import (make_shifted_templates,
                                      make_default_template,
                                      draw_template,
                                      gaus,
                                      make_drift_template,
                                      shift_templates_all,
                                      WaveForms,
                                      generate_poisson_uniform_firingrate2,
                                      select_ground_truth_units,
                                      visualize_drift,
                                      generate_synthetic_data,
                                      visualize_traces,
                                      make_full_data)

colors = [
'black','grey','brown','slategrey',    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
clrs= colors


Autosaving every 180 seconds


In [2]:
# select working directory location
root_dir = '/media/cat/4TBSSD/data/synthetic/run29_olivier_real_drift/'

ctr=0
while True:
    try:
        os.mkdir(root_dir)
        break
    except:
        root_dir = root_dir[:-1]+"_"+str(ctr)+'/'
        print ("Directory exists... appending another val: ", root_dir)
        

# set radius of channels to consider from maximum channel outwards (in micrometers)
radius = 200

# set data information
sample_rate = 30000
rec_len_sec = 400  # seconds

# select ground truth templates to be used
#fname_templates = '/media/cat/12TB/dbox/Dropbox/code/neuropixels/allen_inst_2ndrec/templates_reloaded.npy'
#geom = np.loadtxt('/home/cat/p1_g0_t0.imec0.ap_geom_384chans.txt')
fname_templates = '/media/cat/4TBSSD/data/synthetic/allen_inst_2ndrec/templates_reloaded.npy'
geom = np.loadtxt('/media/cat/4TBSSD/data/synthetic/p1_g0_t0.imec0.ap_geom.txt')


# select largest X templates to be used for injection
n_templates = 100
(units, templates_ground_truth) = select_ground_truth_units(root_dir, 
                                                            fname_templates,
                                                            n_templates,
                                                            geom)

print ("templates ground truth: ", templates_ground_truth.shape)

Total templates avialable;  (575, 101, 384)
Total units selected:  100 , ids:  [420 339 421 348 473 418  62 362 565 239] ...
templates ground truth:  (100, 101, 384)


In [3]:
# visualize the desired shifts on a particular unit
#shifts = np.array([-2, -1.5, -1.0, -0.5, 0])
#shifts = np.arange(0, 1., 0.05)
#shifts = np.hstack((shifts, np.array([0,1.6, 2.3])))

drift_type = 'real' # random_walk, or real
if drift_type == 'monotonic':
    shifts = np.arange(-0.4, 0.4, 0.01)
    #shifts = np.array([-2.0, -1.99, 1.99, 2.0])

elif drift_type == 'random_walk':
    # random walk 
    inter_chunk_time = 2
    shifts = np.random.normal(0,1,size=rec_len_sec//inter_chunk_time)*.05
    print ("shifts: ", shifts)
    plt.plot(np.cumsum(shifts))
    shifts = np.cumsum(shifts)

elif drift_type == 'real':
    shifts = np.load('/media/cat/4TBSSD/data/olivier_drift/drift_probe01.npy').squeeze()

    # clip
    shifts = shifts[:rec_len_sec]
    
    # divide by 46. which is the micron -> channels conversion
    shifts /= 46.
    print (shifts.shape)
    
    
print (shifts)

np.savetxt(root_dir+'/ground_truth/shifts.txt', shifts)

# visualize drifted template for any particular unit 
if False:
    unit_id = 50
    visualize_drift(shifts, 
                    geom, 
                    templates_ground_truth, 
                    unit_id,
                    radius)


(400,)
[-0.01629556 -0.02718271 -0.03613823 -0.04160108 -0.04535544 -0.0479482
 -0.04659734 -0.0308535   0.00583231  0.05276581  0.08896914  0.10407007
  0.10150504  0.0931594   0.09284561  0.1051837   0.11715122  0.11698486
  0.10632319  0.09429655  0.08528425  0.08083487  0.0834094   0.09429655
  0.10963307  0.12309475  0.13210706  0.13559974  0.13166259  0.11875729
  0.10432233  0.09846834  0.09784781  0.09355652  0.08555355  0.07609174
  0.06517989  0.05557965  0.05196191  0.05199165  0.04959991  0.0444894
  0.03912257  0.04313792  0.06512778  0.09694142  0.11904016  0.11812201
  0.09929928  0.07837045  0.06507444  0.06019538  0.05767764  0.05767997
  0.06422552  0.07522156  0.08261025  0.0808531   0.07774398  0.07939336
  0.07678442  0.06234601  0.04806151  0.04954018  0.06576946  0.0831027
  0.08890404  0.08191758  0.07664305  0.08742311  0.11028011  0.13456734
  0.15301187  0.15859655  0.15829894  0.16613555  0.18197045  0.1901068
  0.18468828  0.17638282  0.1751882   0.18260922

In [4]:
# generate synthetic drift data
rec_length_sample_times = rec_len_sec * sample_rate
firingrate_min = 1
firingrate_max = 3

# generate synthetic data
(data_synthetic, windows, spike_train) = generate_synthetic_data(root_dir,
                                         rec_len_sec, 
                                         sample_rate, 
                                         shifts, 
                                         units, 
                                         templates_ground_truth,
                                         radius,
                                         geom,
                                         firingrate_min,
                                         firingrate_max)


print (data_synthetic.shape)

(12000000, 384)
[       0.            30075.18796992    60150.37593985    90225.56390977
   120300.7518797    150375.93984962   180451.12781955   210526.31578947
   240601.5037594    270676.69172932   300751.87969925   330827.06766917
   360902.2556391    390977.44360902   421052.63157895   451127.81954887
   481203.0075188    511278.19548872   541353.38345865   571428.57142857
   601503.7593985    631578.94736842   661654.13533835   691729.32330827
   721804.5112782    751879.69924812   781954.88721805   812030.07518797
   842105.26315789   872180.45112782   902255.63909774   932330.82706767
   962406.01503759   992481.20300752  1022556.39097744  1052631.57894737
  1082706.76691729  1112781.95488722  1142857.14285714  1172932.33082707
  1203007.51879699  1233082.70676692  1263157.89473684  1293233.08270677
  1323308.27067669  1353383.45864662  1383458.64661654  1413533.83458647
  1443609.02255639  1473684.21052632  1503759.39849624  1533834.58646617
  1563909.77443609  1593984.9624060

 inserting unit:  0 [ 98019 111734 116274] [1.02599845 1.0074869  1.07173902]
 inserting unit:  50 [ 94523  99447 102704 103914] [1.07889612 0.97770427 0.93586516 0.96561771]
************


window:  [120300.75187969925, 150375.93984962406] , shift:  -0.045355441087902536  (inter channel units)
 inserting unit:  0 [140607] [1.05539521]
 inserting unit:  50 [124505 132372 141499 142838] [1.00571148 1.01365959 1.02821648 0.89056732]
************


window:  [150375.93984962406, 180451.12781954888] , shift:  -0.04794820182553902  (inter channel units)
 inserting unit:  0 [] []
 inserting unit:  50 [156828] [0.95005353]
************


window:  [180451.12781954888, 210526.31578947368] , shift:  -0.04659734251465699  (inter channel units)
 inserting unit:  0 [192099] [0.95878446]
 inserting unit:  50 [181490 183568 186105 188622 201155 207125] [0.9422786  0.90201096 0.97793159 0.94876473 1.02304866 1.0318033 ]
************


window:  [210526.31578947368, 240601.5037593985] , shift:  -0.0308535

 inserting unit:  0 [1062974 1065328 1071829 1074822] [0.98778661 0.8995018  1.07766786 1.06815277]
 inserting unit:  50 [1059356 1077792] [0.92028838 0.90292688]
************


window:  [1082706.7669172932, 1112781.954887218] , shift:  0.0651798850861863  (inter channel units)
 inserting unit:  0 [1086658 1095669 1111816] [1.06408554 0.92612019 1.00897488]
 inserting unit:  50 [1105012 1105381] [1.02094829 0.95112639]
************


window:  [1112781.954887218, 1142857.142857143] , shift:  0.05557965439829417  (inter channel units)
 inserting unit:  0 [1120165] [0.97404449]
 inserting unit:  50 [] []
************


window:  [1142857.142857143, 1172932.3308270676] , shift:  0.05196191053115904  (inter channel units)
 inserting unit:  0 [1163140 1166523] [1.07046495 1.09580455]
 inserting unit:  50 [1146579 1168924] [1.0072687  1.07380404]
************


window:  [1172932.3308270676, 1203007.5187969925] , shift:  0.05199165067186322  (inter channel units)
 inserting unit:  0 [1185253 12

 inserting unit:  0 [2075878 2082151] [0.96580707 0.98844441]
 inserting unit:  50 [2089492 2097244] [0.91420398 1.08696765]
************


window:  [2105263.157894737, 2135338.3458646615] , shift:  0.11028011455390441  (inter channel units)
 inserting unit:  0 [2107288 2115301 2121585 2126970] [0.93585387 0.99969389 0.95475927 0.9432429 ]
 inserting unit:  50 [2127584 2132551] [0.9752208  1.09687988]
************


window:  [2135338.3458646615, 2165413.5338345864] , shift:  0.1345673399878396  (inter channel units)
 inserting unit:  0 [2151149 2155405 2162724] [1.09176347 1.09181801 0.90968718]
 inserting unit:  50 [2144006 2146790 2151042 2156042 2160603] [0.93651676 0.9989418  0.94484412 0.99253575 0.93058497]
************


window:  [2165413.5338345864, 2195488.7218045113] , shift:  0.15301187232792737  (inter channel units)
 inserting unit:  0 [2189628 2194858] [1.02901261 0.91660383]
 inserting unit:  50 [2175339 2175456 2176278 2192818 2193139 2193373] [1.09988796 0.94718603 0.9

 inserting unit:  0 [3040638 3058792 3062762 3066264] [1.04296107 1.05078381 0.98406813 0.95684375]
 inserting unit:  50 [3042955 3065792 3066724] [1.08463033 1.0275162  0.85515099]
************


window:  [3067669.172932331, 3097744.360902256] , shift:  0.18720905334117255  (inter channel units)
 inserting unit:  0 [3087544] [0.97132448]
 inserting unit:  50 [3068749 3070221 3080196 3088005 3088637 3096157 3097310] [1.05806256 0.92941018 0.92470507 1.00292098 0.96541134 1.07274015
 0.84904459]
************


window:  [3097744.360902256, 3127819.5488721807] , shift:  0.18007668897427012  (inter channel units)
 inserting unit:  0 [3098778 3116865] [0.91843671 0.95000657]
 inserting unit:  50 [3115053 3121846] [1.00135262 0.90067293]
************


window:  [3127819.5488721807, 3157894.736842105] , shift:  0.17434939569857585  (inter channel units)
 inserting unit:  0 [3137136 3141161] [1.03378505 0.90081911]
 inserting unit:  50 [3133494 3139602 3147404] [0.94717249 0.95707262 1.0421380

...

 inserting unit:  0 [7825557 7837031] [0.98792304 1.05242783]
 inserting unit:  50 [7838001 7840116] [1.09028619 0.91916246]
************


window:  [7849624.0601503765, 7879699.248120301] , shift:  0.2850609305068719  (inter channel units)
 inserting unit:  0 [] []
 inserting unit:  50 [7849667 7862207 7865423] [0.91082632 1.0082178  0.97041838]
************


window:  [7879699.248120301, 7909774.436090225] , shift:  0.26829472047219305  (inter channel units)
 inserting unit:  0 [] []
 inserting unit:  50 [7886829 7896496] [1.0563843  1.08075873]
************


window:  [7909774.436090225, 7939849.62406015] , shift:  0.23928358183819307  (inter channel units)
 inserting unit:  0 [7913118 7915475 7915795 7927155 7931272] [1.01795404 0.96022414 1.00282816 0.92267519 0.99435241]
 inserting unit:  50 [7922414 7932635 7933534] [0.99841296 1.00585968 0.97999045]
************


window:  [7939849.62406015, 7969924.812030075] , shift:  0.23125569957771092  (inter channel units)
 inserting unit

 inserting unit:  0 [8753018 8774051] [1.09379624 1.0496187 ]
 inserting unit:  50 [] []
************


window:  [8781954.887218045, 8812030.07518797] , shift:  0.30110483030691987  (inter channel units)
 inserting unit:  0 [8800484] [0.98938346]
 inserting unit:  50 [8789719 8790328 8790575 8796825 8798464 8802800] [1.04521108 0.88403992 0.9211695  1.06515793 0.90389692 0.93247769]
************


window:  [8812030.07518797, 8842105.263157895] , shift:  0.29013366712686556  (inter channel units)
 inserting unit:  0 [8826888 8829900 8841211] [0.90399387 0.90444679 1.02882375]
 inserting unit:  50 [8815710 8815875 8819409 8826739 8827777 8841299] [0.94843798 0.83026711 0.9672386  1.00275204 0.89495692 1.02895025]
************


window:  [8842105.263157895, 8872180.45112782] , shift:  0.2807538117783018  (inter channel units)
 inserting unit:  0 [8842260] [0.88233766]
 inserting unit:  50 [8844122 8846713 8847961 8851873 8863310 8866240 8868053] [1.06674904 1.03402143 0.92798691 0.9458062

 inserting unit:  0 [9666099 9668878 9672702 9673423] [1.02840792 1.02338534 1.03179858 0.87366501]
 inserting unit:  50 [9658133 9670996 9671165 9672241] [0.96090582 0.94508219 0.99179156 0.99064854]
************


window:  [9684210.52631579, 9714285.714285715] , shift:  0.34121019157311094  (inter channel units)
 inserting unit:  0 [9697762] [0.98306061]
 inserting unit:  50 [9686251 9689957 9690574 9698877] [1.03863361 0.98113436 0.93976774 0.94292521]
************


window:  [9714285.714285715, 9744360.90225564] , shift:  0.39694297934308176  (inter channel units)
 inserting unit:  0 [] []
 inserting unit:  50 [9731471 9744183] [0.91192026 0.98863951]
************


window:  [9744360.90225564, 9774436.090225564] , shift:  0.43429337241376303  (inter channel units)
 inserting unit:  0 [9748972 9751717] [0.98839211 0.91086179]
 inserting unit:  50 [9771183] [1.06898129]
************


window:  [9774436.090225564, 9804511.27819549] , shift:  0.440214534986006  (inter channel units)
 i

 inserting unit:  0 [10622391 10640681] [1.03114641 0.94705258]
 inserting unit:  50 [10618707 10622823 10646525] [0.94877196 0.95427117 1.0000853 ]
************


window:  [10646616.541353384, 10676691.729323309] , shift:  0.1889383506137937  (inter channel units)
 inserting unit:  0 [10654967 10667137 10674157] [0.96722068 0.97989285 1.09995656]
 inserting unit:  50 [] []
************


window:  [10676691.729323309, 10706766.917293234] , shift:  0.20859504423993175  (inter channel units)
 inserting unit:  0 [10677785 10701717] [1.0739414  1.04945986]
 inserting unit:  50 [10687647 10693257 10696706] [0.97875933 0.98132685 1.09314996]
************


window:  [10706766.917293234, 10736842.105263159] , shift:  0.25407383616229084  (inter channel units)
 inserting unit:  0 [] []
 inserting unit:  50 [10722081 10722264 10728530 10733565 10735516] [1.0998865  0.9576285  0.91671195 1.01424213 0.88227486]
************


window:  [10736842.105263159, 10766917.293233084] , shift:  0.2933632672

 inserting unit:  0 [11604857 11605243 11606628] [1.09615668 0.83077345 0.87400894]
 inserting unit:  50 [11605990 11607393] [0.96477994 1.03105814]
************


window:  [11609022.556390978, 11639097.744360903] , shift:  0.30421656746714965  (inter channel units)
 inserting unit:  0 [] []
 inserting unit:  50 [11627536 11633592] [0.90852461 1.06137561]
************


window:  [11639097.744360903, 11669172.932330828] , shift:  0.3061534427243416  (inter channel units)
 inserting unit:  0 [11642518 11667187] [0.95917262 0.91501441]
 inserting unit:  50 [] []
************


window:  [11669172.932330828, 11699248.120300753] , shift:  0.3115060087032484  (inter channel units)
 inserting unit:  0 [11670342 11692588] [1.09083955 0.95598423]
 inserting unit:  50 [11692649 11696804 11698946 11699095] [0.91518445 0.90553443 0.88133027 0.91275687]
************


window:  [11699248.120300753, 11729323.308270678] , shift:  0.3146585592291176  (inter channel units)
 inserting unit:  0 [11713237 1

In [10]:
# SAVE DRIFT TIME SERIES
drift = np.zeros((data_synthetic.shape[0]),'float32')
for k in range(len(windows)):
    drift[int(windows[k][0]):int(windows[k][1])]=shifts[k]

fname = root_dir+'/ground_truth/drift.npy'
np.save(fname, drift)


In [11]:
# # visualize the final traces
# visualize_traces(data_synthetic, 0, 5000)


In [12]:
# Make a correlated data chunk of 10 seconds 
# longer takes too long of time; shorter will add more boundary artifcats to recording
time = 10*sample_rate
print ("length of correlated noise snipit: ", time)

fname = root_dir+'correlated_noise.npy'
fname = '/media/cat/4TBSSD/data/synthetic/run18/correlated_noise.npy'
# use ground_truth_templates to make noise
if os.path.exists(fname)==False:
    temps = templates_ground_truth.copy()
    WF = WaveForms(temps.transpose(0,2,1))
    correlated_noise = WF.generate_correlated_noise_add(time)

    print ("DONE MAKING CORRELATED NOISE for 10 sec chunk")
    
    np.save(fname, correlated_noise)
else:
    correlated_noise = np.load(fname)

length of correlated noise snipit:  300000


In [8]:
# blend correlated noise and white noise with injected spike dataset
data_sum = make_full_data(data_synthetic, rec_length_sample_times, 
                          sample_rate, root_dir,
                          correlated_noise)


white noise:  (300000, 384)
time chunk:  0  to  300000
time chunk:  300000  to  600000
time chunk:  600000  to  900000
time chunk:  900000  to  1200000
time chunk:  1200000  to  1500000
time chunk:  1500000  to  1800000
time chunk:  1800000  to  2100000
time chunk:  2100000  to  2400000
time chunk:  2400000  to  2700000
time chunk:  2700000  to  3000000
time chunk:  3000000  to  3300000
time chunk:  3300000  to  3600000
time chunk:  3600000  to  3900000
time chunk:  3900000  to  4200000
time chunk:  4200000  to  4500000
time chunk:  4500000  to  4800000
time chunk:  4800000  to  5100000
time chunk:  5100000  to  5400000
time chunk:  5400000  to  5700000
time chunk:  5700000  to  6000000
time chunk:  6000000  to  6300000
time chunk:  6300000  to  6600000
time chunk:  6600000  to  6900000
time chunk:  6900000  to  7200000
time chunk:  7200000  to  7500000
time chunk:  7500000  to  7800000
time chunk:  7800000  to  8100000
time chunk:  8100000  to  8400000
time chunk:  8400000  to  870000

In [12]:
# # visualize first 10000 time points;
# visualize_traces(data_sum, 0, 10000)


In [9]:
# import numpy as np
# import h5py

# #mystuff = np.random.rand(10,30)

# f = h5py.File(root_dir+'/data.h5', 'w')
# f['rawdata'] = data_sum
# f['geometry'] = geom
# f['drift'] = drift
# f['spike_times'] = spike_train[:,0]
# f['neuron_ids'] = spike_train[:,1]
# f.close()

# REMAKE / UPDATE HDF5 FILE FROM PREVIOUSL SAVED DATA


In [4]:
import numpy as np
import h5py

if True:
    root_dir = '/media/cat/4TBSSD/data/synthetic/run28_olivier_real_drift/'
    geom = np.loadtxt(root_dir + '/geom_384chans.txt')
    drift = np.load(root_dir + '/ground_truth/drift.npy')
    spike_train = np.load(root_dir + '/ground_truth/spike_train_ground_truth.npy')
    templates = np.load(root_dir + '/ground_truth/templates_ground_truth.npy')
    
    # add new fields
    reorder = np.load(root_dir + '/tmp/preprocess/reorder.npy')
    spike_train_detect = np.load(root_dir + '/tmp/block_1/detect/spike_index.npy')

    
    # make max channel information
    max_chans = templates.ptp(1).argmax(1)
    channel_locations = spike_train[:,1].copy()
    for k in np.unique(spike_train[:,1]):
        idx = np.where(spike_train[:,1]==k)[0]
        channel_locations[idx]=max_chans[k]

    # load raw data
    data_sum = np.fromfile(root_dir + '/data_int16.bin','int16')

    f2 = h5py.File(root_dir+'/data.h5', 'w')
    f2['rawdata'] = data_sum
    f2['geometry'] = geom
    f2['ground_truth_drift'] = drift
    f2['ground_truth_spike_times'] = spike_train[:,0]
    f2['ground_truth_neuron_ids'] = spike_train[:,1]
    f2['kilosort2_reorder'] = reorder
    f2['detected_spike_times'] = spike_train_detect[:,0]
    f2['detected_neuron_channels'] = spike_train_detect[:,1]
    f2['templates'] = templates   
    f2['channel_locations'] = channel_locations

    f2.close()

# CHECK RESULTS AFTER KS2 REORDERING

In [24]:
# TEST KS2 RESULTS
import numpy as np
import h5py

sample_rate = 30000

root_dir = '/media/cat/4TBSSD/data/synthetic/run27_monotonic/'
#root_dir = '/media/cat/4TBSSD/data/synthetic/run26_random_walk/'

reorder = np.load(root_dir + '/tmp/preprocess/reorder.npy')
#print (reorder)

fig=plt.figure()
ax1=plt.subplot(1,1,1)

#mystuff = np.random.rand(10,30)
if False:
    fname = root_dir + '/run27_monotonic.h5'
    f = h5py.File(fname, 'r')
    drift = np.float32(f['drift']).copy()
else:
    drift = np.load(root_dir + '/ground_truth/drift.npy')
    


    
x= np.arange(drift.shape[0])/sample_rate
plt.plot(x,drift, c='black', label="Original drift")

# compute ideal drift
idx = np.argsort(drift)#[::-1]
plt.plot(x,drift[idx],c='blue',label='Ideal monotonic reordering')


# do KS2 reorder
reorder = reorder[::-1]
chunk = 1 #* sample_rate #seconds
diffs = []

for k in range(reorder.shape[0]):
    
    # load KS2 reordered chunk value starts and ends
    ks2_trace = reorder[k]*chunk*sample_rate
    start = int(ks2_trace)
    end = int(start + chunk*sample_rate)
    
    # grab chunk from ground truth drift trace
    ks2_trace = drift[start:end]
    
    # make time series offset to beginning of chunk; 
    x = np.arange(chunk*sample_rate)+k*chunk*sample_rate
    plt.plot(x/sample_rate, ks2_trace,c='red')

    # ground truth drift; ideal reordering
    gt_trace = drift[idx][k*sample_rate:(k+chunk)*sample_rate]
    #temp2 = temp
    print (k, ks2_trace, gt_trace, ks2_trace.shape, gt_trace.shape)
    #diff = np.abs(temp1-temp2).mean(0)
    diff = np.sqrt(((ks2_trace-gt_trace)**2).mean())
           #np.sqrt(((predictions - targets) ** 2).mean())
    diffs.append(diff)
    
print (diffs)
plt.plot(x/sample_rate, temp,c='red', label="KS2 monotonic reorder")
plt.legend(fontsize=12, loc=2)
plt.suptitle(root_dir,fontsize=20)
plt.ylabel("Drift in Channels ", fontsize=20)
plt.xlabel("Time (sec) ", fontsize=20)

#ax2=plt.subplot(1,2,2)
plt.title("Distribution of (L2) KS2 errors; mean = "+str(round(np.mean(diffs),4)))
#bin_width = 0.05
#y = np.histogram(diffs,np.arange(-1,1,bin_width))
#plt.plot(y[1][:-1]+bin_width,y[0])

plt.show()

0 [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
1 [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
2 [-0.38 -0.38 -0.38 ... -0.38 -0.38 -0.38] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
3 [-0.36 -0.36 -0.36 ... -0.36 -0.36 -0.36] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
4 [-0.37 -0.37 -0.37 ... -0.37 -0.37 -0.37] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
5 [-0.34 -0.34 -0.34 ... -0.33 -0.33 -0.33] [-0.4  -0.4  -0.4  ... -0.39 -0.39 -0.39] (30000,) (30000,)
6 [-0.36 -0.36 -0.36 ... -0.36 -0.36 -0.36] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
7 [-0.31 -0.31 -0.31 ... -0.31 -0.31 -0.31] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
8 [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
9 [-0.25 -0.25 -0.25 ... -0.25 -0.25 -0.25] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
10

80 [-0.17 -0.17 -0.17 ... -0.17 -0.17 -0.17] [-0.25 -0.25 -0.25 ... -0.25 -0.25 -0.25] (30000,) (30000,)
81 [-0.2 -0.2 -0.2 ... -0.2 -0.2 -0.2] [-0.25 -0.25 -0.25 ... -0.24 -0.24 -0.24] (30000,) (30000,)
82 [-0.25 -0.25 -0.25 ... -0.24 -0.24 -0.24] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
83 [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
84 [-0.21 -0.21 -0.21 ... -0.21 -0.21 -0.21] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
85 [-0.22 -0.22 -0.22 ... -0.22 -0.22 -0.22] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
86 [-0.26 -0.26 -0.26 ... -0.26 -0.26 -0.26] [-0.24 -0.24 -0.24 ... -0.23 -0.23 -0.23] (30000,) (30000,)
87 [-0.22 -0.22 -0.22 ... -0.22 -0.22 -0.22] [-0.23 -0.23 -0.23 ... -0.23 -0.23 -0.23] (30000,) (30000,)
88 [-0.27 -0.27 -0.27 ... -0.27 -0.27 -0.27] [-0.23 -0.23 -0.23 ... -0.23 -0.23 -0.23] (30000,) (30000,)
89 [-0.21 -0.21 -0.21 ... -0.21 -0.21 -0.21] [-0.23 -0.23 -0.

160 [-0.01 -0.01 -0.01 ... -0.01 -0.01 -0.01] [-0.09 -0.09 -0.09 ... -0.09 -0.09 -0.09] (30000,) (30000,)
161 [-0.08 -0.08 -0.08 ... -0.07 -0.07 -0.07] [-0.09 -0.09 -0.09 ... -0.09 -0.09 -0.09] (30000,) (30000,)
162 [-0.1 -0.1 -0.1 ... -0.1 -0.1 -0.1] [-0.09 -0.09 -0.09 ... -0.08 -0.08 -0.08] (30000,) (30000,)
163 [-0.07 -0.07 -0.07 ... -0.07 -0.07 -0.07] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
164 [-0.09 -0.09 -0.09 ... -0.09 -0.09 -0.09] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
165 [-0.19 -0.19 -0.19 ... -0.19 -0.19 -0.19] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
166 [-0.03 -0.03 -0.03 ... -0.03 -0.03 -0.03] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
167 [-0.15 -0.15 -0.15 ... -0.15 -0.15 -0.15] [-0.08 -0.08 -0.08 ... -0.07 -0.07 -0.07] (30000,) (30000,)
168 [-0.14 -0.14 -0.14 ... -0.13 -0.13 -0.13] [-0.07 -0.07 -0.07 ... -0.07 -0.07 -0.07] (30000,) (30000,)
169 [0.01 0.01 0.01 ... 0.01 0.01 0.01] [-0.07 -0.07

240 [0.08 0.08 0.08 ... 0.08 0.08 0.08] [0.07 0.07 0.07 ... 0.07 0.07 0.07] (30000,) (30000,)
241 [0.07 0.07 0.07 ... 0.07 0.07 0.07] [0.07 0.07 0.07 ... 0.07 0.07 0.07] (30000,) (30000,)
242 [0.06 0.06 0.06 ... 0.06 0.06 0.06] [0.07 0.07 0.07 ... 0.07 0.07 0.07] (30000,) (30000,)
243 [-0.04 -0.04 -0.04 ... -0.04 -0.04 -0.04] [0.07 0.07 0.07 ... 0.08 0.08 0.08] (30000,) (30000,)
244 [0.09 0.09 0.09 ... 0.09 0.09 0.09] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
245 [0.07 0.07 0.07 ... 0.07 0.07 0.07] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
246 [0.09 0.09 0.09 ... 0.09 0.09 0.09] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
247 [0.08 0.08 0.08 ... 0.09 0.09 0.09] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
248 [0.11 0.11 0.11 ... 0.11 0.11 0.11] [0.08 0.08 0.08 ... 0.09 0.09 0.09] (30000,) (30000,)
249 [0.06 0.06 0.06 ... 0.07 0.07 0.07] [0.09 0.09 0.09 ... 0.09 0.09 0.09] (30000,) (30000,)
250 [0.08 0.08 0.08 ... 0.08 0.08 0.08] [0.09 0.09 0.0

330 [0.29 0.29 0.29 ... 0.29 0.29 0.29] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
331 [0.28 0.28 0.28 ... 0.29 0.29 0.29] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
332 [0.34 0.34 0.34 ... 0.34 0.34 0.34] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
333 [0.2 0.2 0.2 ... 0.2 0.2 0.2] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
334 [0.28 0.28 0.28 ... 0.28 0.28 0.28] [0.25 0.25 0.25 ... 0.26 0.26 0.26] (30000,) (30000,)
335 [0.22 0.22 0.22 ... 0.22 0.22 0.22] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
336 [0.23 0.23 0.23 ... 0.24 0.24 0.24] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
337 [0.23 0.23 0.23 ... 0.23 0.23 0.23] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
338 [0.32 0.32 0.32 ... 0.32 0.32 0.32] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
339 [0.24 0.24 0.24 ... 0.25 0.25 0.25] [0.26 0.26 0.26 ... 0.27 0.27 0.27] (30000,) (30000,)
340 [0.24 0.24 0.24 ... 0.24 0.24 0.24] [0.27 0.27 0.27 ... 0.27 0

In [2]:
# Load previously saved dataset
root_dir = '/media/cat/4TBSSD/data/synthetic/run27_monotonic/'

f = h5py.File(root_dir+'/data.h5', 'r')
print (f.keys())
drift = np.float32(f.get('drift'))
data_sum = np.array(f.get('rawdata'))
geom = np.array(f.get('geometry'))
spike_times = np.array(f.get('spike_times'))
spike_ids = np.array(f.get('neuron_ids'))
f.close()

print (data_sum.shape)



<KeysViewHDF5 ['drift', 'geometry', 'neuron_ids', 'rawdata', 'spike_times', 'templates']>
(4608000000,)


In [3]:
# add new fields
reorder = np.load(root_dir + '/tmp/preprocess/reorder.npy')
spike_train_detect = np.load(root_dir + '/tmp/block_1/detect/spike_index.npy')


In [ ]:
# add new fileds

In [4]:
f2 = h5py.File(root_dir+'/data_updated.h5', 'w')
f2['rawdata'] = data_sum
f2['geometry'] = geom
f2['ground_truth_drift'] = drift
f2['ground_truth_spike_times'] = spike_times
f2['ground_truth_neuron_ids'] = spike_ids
f2['kilosort2_reorder'] = reorder
f2['detected_spike_times'] = spike_train_detect[:,0]
f2['detected_neuron_channels'] = spike_train_detect[:,1]
f2.close()

In [8]:
events = np.load('/media/cat/4TBSSD/data/synthetic/run25_random_walk/tmp/block_1/detect/spike_index.npy')


In [5]:
data = np.load('/media/cat/4TBSSD/data/olivier_drift/drift_probe00.npy')
plt.plot(data/46.)
plt.show()

In [22]:
# add channels
root_dir = '/media/cat/4TBSSD/data/synthetic/run27_monotonic/'
spike_train = np.load(root_dir + '/ground_truth/spike_train_ground_truth.npy')
templates = np.load(root_dir +'/ground_truth/templates_ground_truth.npy')
print (templates.shape)

max_chans = templates.ptp(1).argmax(1)
channel_locations = spike_train[:,1].copy()
for k in np.unique(spike_train[:,1]):
    idx = np.where(spike_train[:,1]==k)[0]
    channel_locations[idx]=max_chans[k]
    
print (channel_locations[:40])
print (spike_train[:40,1])
f2['channel_locations'] = channel_locations


(100, 101, 384)
[251 251 251 251 251 251 251 251 251 251 251 251 251 251 251 251 251 251
 251 182 182 182 182 182 182 182 182 182 182 182 182 182 182 182 182 182
 182 182 182 182]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [23]:
f2 = h5py.File(root_dir+'/data_updated_channels.h5', 'w')

f2['channel_locations'] = channel_locations
f2.close()

In [25]:
print (spike_train.shape)

(123858, 2)


In [12]:
pos = np.load('/media/cat/4TBSSD/Downloads/Copy of Copy of manip.positions.npy')/46.
data = np.load('/media/cat/4TBSSD/Downloads/Copy of Copy of manip.timestamps_p2.npy')
ax=plt.subplot(121)
plt.plot(data, pos)
plt.title("NP1 probe")
plt.ylabel("Drift (channel units)")
plt.xlabel("Time (sec)")

data2 = np.load('/media/cat/4TBSSD/Downloads/Copy of Copy of manip.timestamps_p1.npy')
ax=plt.subplot(122)
plt.title("NP2 probe")
plt.xlabel("Time (sec)")
plt.plot(data2, pos,c='black')
plt.show()